In [1]:
# import
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("TV Shows - Association Rule Learning.csv", header=None)

from collections import defaultdict


In [2]:
# importing dataset ( example 1 and example 2 are datasets in pyECLAT)
#from pyECLAT import Example2
# Load dataset

df.info()
# storing the dataset in a variable
#dataset = df.get()
# printing the dataset
df.head()
#dataset = df.get()
# Create a copy of the first row
first_row_copy = df.iloc[0:1].copy()

# Append the copy as the last row
df = pd.concat([df, first_row_copy], ignore_index=True)
#num_columns = len(df.columns)
#df.columns = [f"Column{i}" for i in range(1, num_columns + 1)]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9690 entries, 0 to 9689
Data columns (total 32 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       9690 non-null   object
 1   1       7557 non-null   object
 2   2       5947 non-null   object
 3   3       4667 non-null   object
 4   4       3678 non-null   object
 5   5       2837 non-null   object
 6   6       2197 non-null   object
 7   7       1664 non-null   object
 8   8       1231 non-null   object
 9   9       887 non-null    object
 10  10      644 non-null    object
 11  11      464 non-null    object
 12  12      348 non-null    object
 13  13      272 non-null    object
 14  14      196 non-null    object
 15  15      141 non-null    object
 16  16      95 non-null     object
 17  17      66 non-null     object
 18  18      52 non-null     object
 19  19      38 non-null     object
 20  20      29 non-null     object
 21  21      18 non-null     object
 22  22      14 non-null     

In [3]:
# numpy array of items
# importing the ECLAT module
from pyECLAT import ECLAT
# loading watch DataFrame to ECLAT class
eclat = ECLAT(data=df)
# DataFrame of binary values
eclat.df_bin

,Shooter,Madmen,Banshee,The Good Doctor,Into the night,Upload,Westworld,His Dark Materials,Shadow and Bone,South Park,...,Constantine,Elementary,The Mentalist,Gotham,Daredevil,Narcos,Teen Wolf,Good Omens,Friends,How I met your mother
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9686,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9687,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9688,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9689,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# count items in each column
items_total = eclat.df_bin.astype(int).sum(axis=0)
items_total

Shooter                  105
Madmen                   109
Banshee                  370
The Good Doctor           55
Into the night            80
                        ... 
Narcos                    32
Teen Wolf                  7
Good Omens                 6
Friends                   55
How I met your mother    779
Length: 156, dtype: int64

In [5]:
# count items in each row
items_per_watch = eclat.df_bin.astype(int).sum(axis=1)
items_per_watch

0        4
1       12
2        4
3        6
4        1
        ..
9686     7
9687     3
9688     3
9689     1
9690     4
Length: 9691, dtype: int64

In [6]:
eclat.df_bin.astype(int).sum(axis=1)

0        4
1       12
2        4
3        6
4        1
        ..
9686     7
9687     3
9688     3
9689     1
9690     4
Length: 9691, dtype: int64

In [7]:
# Loading items per column stats to the DataFrame
df = pd.DataFrame({'items': items_total.index, 'watch': items_total.values}) 
# cloning pandas DataFrame for visualization purpose  
df_table = df.sort_values("watch", ascending=False)
#  Top 5 most popular products/items
df_table.head(5).style.background_gradient(cmap='Blues')

,items,watch
61,Sex Education,2477
68,Ozark,1877
16,Two and a half men,1779
130,Outer Banks,1696
97,Atypical,1356


In [8]:
# importing required module
import plotly.express as px
# to have a same origin
df_table["all"] = "Tree Map" 
# creating tree map using plotly
fig = px.treemap(df_table.head(50), path=['all', "items"], values='watch',
                  color=df_table["watch"].head(50), hover_data=['items'],
                  color_continuous_scale='Blues',
                )
# ploting the treemap
fig.show()

In [9]:
# the item shoud appear at least at 5% of watch
min_support = 5/100
# start from watch containing at least 2 items
min_combination = 2
# up to maximum items per watch
max_combination = max(items_per_watch)
rule_indices, rule_supports = eclat.fit(min_support=min_support,
                                                 min_combination=min_combination,
                                                 max_combination=max_combination,
                                                 separator=' & ',
                                                 verbose=True)

Combination 2 by 2


406it [00:01, 397.95it/s]


Combination 3 by 3


3654it [00:08, 422.84it/s]


Combination 4 by 4


...

118755it [05:00, 395.30it/s]


Combination 6 by 6


475020it [21:40, 365.23it/s]


Combination 7 by 7


1261619it [58:34, 366.58it/s]

In [ ]:
result = pd.DataFrame(rule_supports.items(),columns=['Item', 'Support'])
result.sort_values(by=['Support'], ascending=False)